# Modelos para Previsão de Gastos em Saúde

Na primeira parte desse estudo seguimos os critérios do desafio proposto no [FreeCodeCamp](https://www.freecodecamp.org/learn/machine-learning-with-python/machine-learning-with-python-projects/linear-regression-health-costs-calculator).

Nesse notebook vamos estudar outros modelos de previsão para prever os gastos em saúde

# 1.Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor
)
from sklearn.impute import SimpleImputer
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import (
    LassoCV, RidgeCV, ElasticNetCV, SGDRegressor
)
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    StandardScaler, OneHotEncoder
)
from sklearn.tree import DecisionTreeRegressor
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam

# Formatting Table Outputs
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

# 2.Importando dataset e informações gerais

In [2]:
df = pd.read_csv('data/gastos_saude.csv')

In [3]:
df.drop_duplicates(inplace=True)

# 3.Pré-processamento

## Train test split

In [4]:
X = df[['bmi', 'age', 'smoker']]
Y = df['expenses']

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42
)

## Transformações e encodings

In [5]:
numericas = X_train.select_dtypes('number').columns
categoricas = X_train.select_dtypes('object').columns

In [6]:
processador_num = Pipeline([('s_scaler', StandardScaler())])

processador_cat = Pipeline([('ohe', OneHotEncoder(drop="if_binary"))])

preprocessador = ColumnTransformer(transformers = [
    ("processador_num_1", processador_num, numericas),
    ("processador_cat_1", processador_cat, categoricas),
    ], remainder='passthrough', verbose_feature_names_out=False)

In [7]:
X_train_preproc = pd.DataFrame(
    preprocessador.fit_transform(X_train),
    columns = preprocessador.get_feature_names_out()
)

X_test_preproc = pd.DataFrame(
    preprocessador.transform(X_test),
    columns = preprocessador.get_feature_names_out()
)

# 4.Modelagem

Aqui vamos utilizar diversos modelos para prever os gastos, e para facilitar nosso trabalho vamos criar uma função que itera pelos itens de um dicionário, armazena os dados obtidos e cria um dataframe para analisarmos as métricas obtidas de cada modelo.

Temos diversas opções para modelagem, e dentre elas vamos usar:
- __Ridge__: o objetivo é minimizar o erro quadrático somado a uma penalização da soma dos coeficientes ao quadrado. Isso ajuda a evitar overfitting e reduzir a influência de coeficientes menos relevantes.

- __Lasso__: o objetivo é minimizar o erro quadrático somado a uma penalização da soma dos valores absolutos dos coeficientes. Isso ajuda a evitar overfitting e a selecionar automaticamente as variáveis mais relevantes.

- __Elastic Net__: aqui combinamos a penalização de Ridge e Lasso em uma única regressão, com um parâmetro de mistura que controla o grau de ambas as penalizações. Isso ajuda a lidar com problemas em que há multicolinearidade e muitas variáveis irrelevantes.

- __Gradiente Descendente__: aqui queremos minimizar a função de perda (como o erro quadrático) usando o método de gradiente descendente, que ajusta os coeficientes iterativamente para minimizar a função de perda. A versão estocástica usa amostras aleatórias do conjunto de treinamento para atualizar os coeficientes, o que torna o processo mais rápido e escalável para conjuntos de dados grandes.

- __Decision Tree Regressor__: dividimos recursivamente os dados em subconjuntos com base em variáveis e valores de corte que maximizam a pureza dos subconjuntos em relação à variável de destino. Isso cria uma estrutura de árvore de decisão que pode ser usada para prever a variável de destino para novos dados.

- __Random Forest Regressor__: nesse caso criamos um conjunto de árvores de decisão usando amostras aleatórias do conjunto de treinamento e variáveis aleatórias para dividir os nós da árvore. Isso ajuda a reduzir a variância e o overfitting em comparação com uma única árvore de decisão.

- __Gradient Boosting Regressor__: nesse modelo é criado conjunto de árvores de decisão que sejam ajustadas iterativamente aos resíduos da árvore anterior, minimizando a função de perda (como o erro quadrático). Isso ajuda a melhorar a precisão da previsão e a reduzir o overfitting.

- __MultiLayer Perceptron__: este é um modelo extremamente poderoso, baseado em redes neurais, no qual o objetivo é criar uma rede neural artificial com várias camadas de neurônios interconectados, com o objetivo de prever uma variável de destino. Cada camada usa uma função de ativação para transformar a entrada em uma saída e a saída é propagada para a próxima camada. O MLP usa backpropagation para ajustar os pesos da rede para minimizar a função de perda (como o erro quadrático).

- __Radial Basis Function__: outro modelo utilizando redes neurais no qual o objetivo é criar uma rede neural artificial que usa funções de base radial para transformar as entradas em uma saída. As funções de base radial são centradas em pontos específicos no espaço de entrada e usam a distância Euclidiana entre os pontos de entrada e o centro para determinar a saída da função. A rede usa backpropagation para ajustar os pesos da rede para minimizar a função de perda (como o erro quadrático).

In [8]:
# Criando dicionário
dic_modelos = {
    'RidgeCV Regression':RidgeCV(cv=5, 
                                 alphas=[0.001, 0.01, 0.1, 1, 10, 100]),
    'LassoCV Regression':LassoCV(random_state=42), 
    'Elastic Net':ElasticNetCV(random_state=42),
    'Gradiente Descendente':SGDRegressor(random_state=42),
    'Decision Tree Regressor':DecisionTreeRegressor(max_depth=3, 
                                                     random_state=42),
    'Random Forest Regressor':RandomForestRegressor(max_depth=3, 
                                                     random_state=42),
    'Gradient Boosting Regressor':GradientBoostingRegressor(random_state=42, 
                                                            learning_rate=0.05),
    'MultiLayer Perceptron':MLPRegressor(hidden_layer_sizes=(100, 50), 
                                         max_iter=100, learning_rate_init=0.03),
    'Radial Basis Function':KernelRidge(kernel='rbf'),
}

In [9]:
# Criando função para resumir modelos
def resumo_modelos(dic, X_treino, y_treino, X_teste, y_teste):
    lista_nome = []
    lista_score = []
    lista_mse = []
    lista_mae = []
    for nome_modelo, modelo in dic.items():
        modelo.fit(X_treino, y_treino)
        y_pred = modelo.predict(X_teste)
        modelo_score = modelo.score(X_treino, y_treino).round(3)
        modelo_mse = metrics.mean_squared_error(y_teste, y_pred).round(3)
        modelo_mae = metrics.mean_absolute_error(y_teste, y_pred).round(3)
        lista_nome.append(nome_modelo)
        lista_score.append(modelo_score)
        lista_mse.append(modelo_mse)
        lista_mae.append(modelo_mae)
    resumo_modelos = pd.DataFrame({'Modelo':lista_nome, 'R² Score':lista_score,
                                   'MSE':lista_mse, 'MAE':lista_mae})
    resumo_modelos.sort_values(by='R² Score', ascending=False, inplace=True, ignore_index=True)
    return resumo_modelos

In [10]:
df_modelos = resumo_modelos(dic_modelos, X_train_preproc, y_train, X_test_preproc, y_test)

In [11]:
df_modelos

,Modelo,R² Score,MSE,MAE
0,Gradient Boosting Regressor,0.873,22162121.230,2565.615
1,Random Forest Regressor,0.855,22562601.276,2731.002
2,Decision Tree Regressor,0.848,24462234.426,2970.526
3,Radial Basis Function,0.836,28058095.089,2987.111
4,MultiLayer Perceptron,0.819,26214369.939,2930.532
5,RidgeCV Regression,0.730,39401832.308,4256.858
6,LassoCV Regression,0.730,39281490.177,4244.300
7,Gradiente Descendente,0.730,39319921.954,4252.444
8,Elastic Net,0.092,154786308.480,9057.894


## Rede neural (Keras/Tensorflow)

In [12]:
X = df.drop(columns = ['expenses'])
Y = df['expenses']

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42
)

In [13]:
num_cols = X_train.select_dtypes('number').columns
cat_cols = X_train.select_dtypes(exclude = 'number').columns

In [14]:
preprocessamento_numericas = Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('escala_features',StandardScaler())
])
preprocessamento_categoricas = Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(drop='if_binary'))
])

In [15]:
preprocessador_final = ColumnTransformer(
    transformers=[
        ('processamento_numericas',preprocessamento_numericas,num_cols),
        ('processamento_categoricas',preprocessamento_categoricas,cat_cols),
        ],remainder='passthrough',verbose_feature_names_out=False
)
                                    
dados_treino_prep = pd.DataFrame(
    preprocessador_final.fit_transform(X_train),
    columns=preprocessador_final.get_feature_names_out()
)

dados_teste_prep = pd.DataFrame(
    preprocessador_final.fit_transform(X_test),
    columns=preprocessador_final.get_feature_names_out()
)

In [16]:
train_samples = np.array(dados_treino_prep).astype('float32')
train_labels = np.array(y_train).astype('float32')

test_samples = np.array(dados_teste_prep).astype('float32')
test_labels = np.array(y_test).astype('float32')

In [17]:
model = Sequential([
    Dense(units=16, input_shape=(9,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=1, activation='linear')
])

In [18]:
model.compile(optimizer=Adam(learning_rate=0.05), 
              loss='mean_squared_error', metrics=['mae'])

In [19]:
model.fit(x=train_samples, y=train_labels, 
          batch_size=10, validation_split=0.1, 
          epochs=50, shuffle=True, verbose=True)

Epoch 1/50
85/85 [==============================] - 2s 7ms/step - loss: 184271456.0000 - mae: 9485.4033 - val_loss: 78762696.0000 - val_mae: 5774.5088
Epoch 2/50
85/85 [==============================] - 0s 4ms/step - loss: 54118428.0000 - mae: 5660.7563 - val_loss: 39965780.0000 - val_mae: 4613.0840
Epoch 3/50
85/85 [==============================] - 0s 2ms/step - loss: 37889636.0000 - mae: 4250.9170 - val_loss: 34548548.0000 - val_mae: 3825.1121
Epoch 4/50
85/85 [==============================] - 0s 2ms/step - loss: 35957296.0000 - mae: 4063.5164 - val_loss: 35018632.0000 - val_mae: 4310.6973
Epoch 5/50
85/85 [==============================] - 0s 2ms/step - loss: 35362336.0000 - mae: 4249.4146 - val_loss: 32003402.0000 - val_mae: 3648.7837
Epoch 6/50
85/85 [==============================] - 0s 3ms/step - loss: 32892522.0000 - mae: 3763.2825 - val_loss: 31405656.0000 - val_mae: 4115.0015
Epoch 7/50
85/85 [==============================] - 0s 2ms/step - loss: 32863364.0000 - mae: 3824.3

In [20]:
predictions = model.predict(x=test_samples, batch_size=10, verbose=0)

In [21]:
model_mse, model_mae = model.evaluate(x=test_samples, y=test_labels)

13/13 [==============================] - 0s 1ms/step - loss: 26081150.0000 - mae: 2679.7524


In [22]:
model_score = metrics.r2_score(test_labels, predictions)

In [23]:
print('=' * 46)
print('Redes Neurais (Keras/TensorFlow)')
print('=' * 46)
print(f'R²: {model_score.round(3)}')
print('-' * 46)
print(f"Erro quadrático médio: {round(model_mse, 3)}")
print('-' * 46)
print(f"Erro absoluto médio: {round(model_mae, 3)}")
print('=' * 46)

Redes Neurais (Keras/TensorFlow)
R²: 0.848
----------------------------------------------
Erro quadrático médio: 26081150.0
----------------------------------------------
Erro absoluto médio: 2679.752


# 5.Conclusão

In [24]:
rede_neural = pd.DataFrame({'Modelo':'Rede Neural (Keras/TF)', 
                            'R² Score':round(model_score,3), 'MSE':round(model_mse,3), 
                            'MAE':round(model_mae,3)}, index=[0])

modelos_sumario = pd.concat([df_modelos, rede_neural], ignore_index=True)
modelos_sumario = modelos_sumario.sort_values(by='MAE', ascending=True)

In [25]:
modelos_sumario

,Modelo,R² Score,MSE,MAE
0,Gradient Boosting Regressor,0.873,22162121.230,2565.615
9,Rede Neural (Keras/TF),0.848,26081150.000,2679.752
1,Random Forest Regressor,0.855,22562601.276,2731.002
4,MultiLayer Perceptron,0.819,26214369.939,2930.532
2,Decision Tree Regressor,0.848,24462234.426,2970.526
3,Radial Basis Function,0.836,28058095.089,2987.111
6,LassoCV Regression,0.730,39281490.177,4244.300
7,Gradiente Descendente,0.730,39319921.954,4252.444
5,RidgeCV Regression,0.730,39401832.308,4256.858
8,Elastic Net,0.092,154786308.480,9057.894


Conseguimos implementar modelos mais complexos e com melhores métricas para fazer a previsão de gastos, sendo o melhor deles o Gradient Boosting Regressor. Porém devemos ter cuidado pois alguns modelos podem tender mais facilmente ao overfitting e são computacionalmente mais caros e devemos analisar essas situações com os diagnósticos de modelo mostrados no primeiro notebook.

Então se tivermos um modelo simples que atenda as nossas expectativas e resolva nosso problema de forma satisfatória sem sombra de dúvidas podemos utiliza-lo ganhando também em explicabilidade e agilidade.

# Referências

- https://towardsdatascience.com/implement-gradient-descent-in-python-9b93ed7108d1

- https://scikit-learn.org/stable/user_guide.html

- https://www.tensorflow.org/tutorials/keras/regression?hl=pt-br